# Kalman Filter (KF)

The KF is one of the most widely used algorithms in time-series applications, such as tracking objects or state estimation in robot navigation. In this notebook, we'll provide a brief overview of the main theory underlying the KF and then demonstrate its utility in an application where we track an object moving in $\mathbb{R}^3$ space, given noisy measurements of its location.

Below are the necessary imports for this notebook:

In [ ]:
# Widget to manipulate plots in Jupyter notebooks
%matplotlib widget 

import matplotlib.pyplot as plt # For general plotting

# GPU suitable form of NumPy, also works on the CPU
from jax import numpy as jnp
from jax import random as jr
from jax import vmap

# JAX library for SSMs
from dynamax.utils.plotting import plot_uncertainty_ellipses
from dynamax.linear_gaussian_ssm import LinearGaussianSSM
from dynamax.linear_gaussian_ssm import lgssm_smoother, lgssm_filter

jnp.set_printoptions(suppress=True)

# Set seed to generate reproducible "pseudo-randomness"
jnp.random.seed(7)

plt.rc('font', size=22)          # controls default text sizes
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=18)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
plt.rc('legend', fontsize=16)    # legend fontsize
plt.rc('figure', titlesize=22)   # fontsize of the figure title

<b>Aside:</b> please note the inclusion of [JAX](https://github.com/google/jax) and [Dynamax](https://github.com/probml/dynamax). JAX is akin to NumPy in syntax and use, but processes instructions on the GPU, efficiently computes gradients (autograd), and reaps the benefits of [just-in-time (JIT) compilation](https://en.wikipedia.org/wiki/Just-in-time_compilation) (e.g., [Numba](https://numba.pydata.org/)), which are all advantageous features for machine learning coding. 

Dynamax is a library written in JAX for <b>state space models (SSMs)</b>. The KF is an algorithm that applies to a special type of SSM: a <b>Linear-Gaussian SSM (LG-SSM)</b>. 

You should follow the [installation instructions](https://github.com/google/jax#installation) for JAX (feel free to use CPU/GPU in this notebook). You can also refer [here](https://github.com/cloudhan/jax-windows-builder) if you're a Windows user.

## Bayesian Filtering & Smoothing

Before proceeding with 